# Tarea 2: Unit Commitment
## Pruebas

In [21]:
using DataFrames, XLSX
using JuMP , HiGHS      # https://jump.dev/JuMP.jl/stable/manual/models/ 
using Gurobi

Leer archivo excel

In [38]:
xf = XLSX.readxlsx("Case014.XLSX")

XLSXFile("Case014.XLSX") containing 5 Worksheets
            sheetname size          range        
-------------------------------------------------
                Buses 22x5          A1:E22       
               Demand 19x51         A1:AY19      
           Generators 11x20         A1:T11       
                Lines 22x7          A1:G22       
           Renewables 30x25         A1:Y30       


Nombres de cada hoja

In [39]:
XLSX.sheetnames(xf)[1:5]

5-element Vector{String}:
 "Buses"
 "Demand"
 "Generators"
 "Lines"
 "Renewables"

In [40]:
XLSX.sheetnames(xf)[2]

"Demand"

In [41]:
length(XLSX.sheetnames(xf))

5

Guardar hojas en lugares distintos

In [42]:
buses = xf["Buses"]
demanda = xf["Demand"]
generadores = xf["Generators"]
lineas = xf["Lines"]
renovables = xf["Renewables"] 

30×25 XLSX.Worksheet: ["Renewables"](A1:Y30) 

Acceder a datos

In [27]:
println(renovables["B3"])   # B3
println(renovables[3,2])    # fila3 , columna2(B)

31.167028084846212
31.167028084846212


In [28]:
renovables["A2:D4"]

3×4 Matrix{Any}:
 "Gen/Hour"   1       2        3
 "Wind2"     31.167  29.4256  27.6723
 "Solar8"     0.0     0.0      0.0

In [29]:
df3 = DataFrame(renovables["A2:D4"])

ArgumentError: ArgumentError: `DataFrame` constructor from a `Matrix` requires passing :auto as a second argument to automatically generate column names: `DataFrame(matrix, :auto)`

Funcion de detencion (en "END")

In [30]:
# Definir la función de parada
function stop_condition(row)
    return isempty(row[1]) || row[1] == "END"
end

stop_condition (generic function with 1 method)

### Leer tablas de hojas de datos

Buses

In [31]:
df1 = DataFrame(XLSX.gettable(buses,"A:B",stop_in_row_function=stop_condition))

Row,Bus,Vmax [pu]
,Any,Any
1,Bus1,1.06
2,Bus2,1.06
3,Bus3,1.06
4,Bus4,1.06
5,Bus5,1.06
6,Bus6,1.06
7,Bus7,1.06
8,Bus8,1.06
9,Bus9,1.06


In [32]:
df1[1,1]

"Bus1"

In [35]:
typeof(df1[1,2])

Float64

In [34]:
parse(Float64, df1[1,1])

ArgumentError: ArgumentError: cannot parse "Bus1" as Float64

Caso de demanda reactiva (Q)

In [90]:
df2 = DataFrame(XLSX.gettable(demanda,"AA:AY",first_row=2,stop_in_row_function=stop_condition))

Row,Bus/Hour,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,Bus1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bus2,12.7,10.3561,9.10085,8.15938,7.84556,9.41467,10.9838,12.2391,12.8667,13.8082,13.9651,13.1805,12.5529,11.9253,13.8082,14.122,13.3375,13.9651,14.7497,15.3773,15.6911,14.122,13.6513,12.8667
3,Bus3,19.0,15.4934,13.6154,12.2069,11.7375,14.0849,16.4324,18.3104,19.2494,20.6579,20.8927,19.7189,18.7799,17.8409,20.6579,21.1274,19.9537,20.8927,22.0664,23.0054,23.4749,21.1274,20.4232,19.2494
4,Bus4,-3.9,-3.18023,-2.79475,-2.50564,-2.40927,-2.89112,-3.37297,-3.75846,-3.9512,-4.24031,-4.28849,-4.04757,-3.85483,-3.66208,-4.24031,-4.33668,-4.09575,-4.28849,-4.52942,-4.72216,-4.81853,-4.33668,-4.19212,-3.9512
5,Bus5,1.6,1.30471,1.14656,1.02795,0.988417,1.1861,1.38378,1.54193,1.621,1.73961,1.75938,1.66054,1.58147,1.50239,1.73961,1.77915,1.68031,1.75938,1.85822,1.9373,1.97683,1.77915,1.71985,1.621
6,Bus6,7.5,6.11583,5.37452,4.81853,4.6332,5.55985,6.48649,7.2278,7.59846,8.15444,8.2471,7.78378,7.41313,7.04247,8.15444,8.33977,7.87645,8.2471,8.71042,9.08108,9.26641,8.33977,8.06178,7.59846
7,Bus7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Bus8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Bus9,16.6,13.5364,11.8956,10.665,10.2548,12.3058,14.3568,15.9975,16.8179,18.0485,18.2536,17.2281,16.4077,15.5873,18.0485,18.4587,17.4332,18.2536,19.2791,20.0995,20.5097,18.4587,17.8434,16.8179


In [110]:
df2[2,1]

"Bus2"

generadores

In [43]:
df3 = DataFrame(XLSX.gettable(generadores,"A:T",first_row=1,stop_in_row_function=stop_condition))

Row,Generator,Bus,Pmax [MW],Pmin [MW],Qmax [MVAR],Qmin [MVAR],Ramp [MW/h],SRamp [MW],MinUP,MinDW,InitS,InitP,StartUpCost [$],FixedCost [$],VariableCost [$/MWh],Type,PminFactor,QFactor,RampFactor,StartUpCostFactor
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,G1,1,100,40,70,-70,20,40.01,8,8,-8,0,3000,600,20,Coal,0.4,0.7,0.2,30
2,G2,2,75,7.5,52.5,-52.5,22.5,22.5,4,4,-4,0,2250,450,40,Gas,0.1,0.7,0.3,30
3,G3,3,75,7.5,52.5,-52.5,22.5,22.5,4,4,-4,0,2250,450,60,Gas,0.1,0.7,0.3,30
4,G6,6,75,7.5,52.5,-52.5,22.5,22.5,4,4,-4,0,2250,450,80,Gas,0.1,0.7,0.3,30
5,G8,8,75,7.5,52.5,-52.5,22.5,22.5,4,4,-4,0,2250,450,100,Gas,0.1,0.7,0.3,30
6,Wind2,2,34.3074,0,24.0152,-24.0152,343.074,343.074,1,1,-1,0,0,0,0,Wind,0.0,0.7,10.0,0
7,Solar8,8,36.0918,0,25.2643,-25.2643,360.918,360.918,1,1,-1,0,0,0,0,Solar,0.0,0.7,10.0,0


In [112]:
df3[:,1] # nombres

7-element Vector{Any}:
 "G1"
 "G2"
 "G3"
 "G6"
 "G8"
 "Wind2"
 "Solar8"

In [45]:
typeof(df3[1,2])

Int64

In [ ]:
typeof()

Renovables

In [106]:
df4 = DataFrame(XLSX.gettable(renovables,"A:Y",first_row=2,stop_in_row_function=stop_condition))

Row,Gen/Hour,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,Wind2,31.167,29.4256,27.6723,28.6323,28.7513,29.2233,28.6362,29.739,30.4213,31.766,33.1306,36.4072,38.4619,40.5286,41.3934,42.516,45.4951,42.6112,37.7479,34.4872,35.9827,35.471,32.5038,31.2067
2,Solar8,0.0,0.0,0.0,0.0,0.000148093,7.14521,32.1157,64.6244,84.5466,90.8603,92.6444,93.0311,92.748,91.194,86.7143,73.3265,41.8975,14.7328,0.62214,0.0,0.0,0.0,0.0,0.0


In [ ]:
df4["1"]

Lineas

In [113]:
df5 = DataFrame(XLSX.gettable(lineas,"A:G",first_row=1,stop_in_row_function=stop_condition))

Row,Branch Name,FromBus,ToBus,Resistance (R),Reactance (X),LineCharging (B),Max Flow [MW]
,Any,Any,Any,Any,Any,Any,Any
1,Line1To2,Bus1,Bus2,0.01938,0.05917,0,135
2,Line1To5,Bus1,Bus5,0.05403,0.22304,0,135
3,Line2To3,Bus2,Bus3,0.04699,0.19797,0,135
4,Line2To4,Bus2,Bus4,0.05811,0.17632,0,135
5,Line2To5,Bus2,Bus5,0.05695,0.17388,0,135
6,Line3To4,Bus3,Bus4,0.06701,0.17103,0,135
7,Line4To5,Bus4,Bus5,0.01335,0.04211,0,135
8,Line4To7,Bus4,Bus7,0,0.20912,0,135
9,Line4To9,Bus4,Bus9,0,0.55618,0,135


# Modelo JuMP

In [12]:
### Problema optimizacion
model = Model(Gurobi.Optimizer) # Crear objeto "modelo" con el solver Gurobi

## Variables
I = 20
T = 24
N = 5
# Variables Economic Dispatch
@variable(model, p[1:I, 1:T] >= 0)  # Potencia activa de generador i en tiempo t. Valor en p.u.
@variable(model, d[1:N, 1:T])       # angulo (d de degree) de la barra n en tiempo t

# Variables nuevas (UC)
# No se utiliza potencia reactiva: Aproximacion DC
#@variable(model, q[1:I, 1:T])       # Potencia reactiva de generador i en tiempo t. Valor en p.u. (puede ser < 0)
@variable(model, u[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Indica encendido de gen i en t.
@variable(model, v[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Indica apagado de gen i en t.
@variable(model, w[1:I, 1:T], Bin)       # AGREGAR NATURALEZA {0,1}. Estado ON(1)/OFF(0) de gen i en t.

# Variables BESS (no se utilizan)
#@variable(model, pb[1:B, 1:T])      # flujo de potencia del BESS b en tiempo t
#@variable(model, e[1:B,1:T] >=0)    # energia almacenada en el BESS e en tiempo t

## Funcion Objetivo
#@objective(model, Min, sum(Gen[i].VariableCostCost * p[i,t] +  Gen[i].FixedCost * w[i,t] + Gen[i].StartCost * u[i,t] for i in 1:I, t in 1:T ))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-26


20×24 Matrix{VariableRef}:
 w[1,1]   w[1,2]   w[1,3]   w[1,4]   w[1,5]   …  w[1,22]   w[1,23]   w[1,24]
 w[2,1]   w[2,2]   w[2,3]   w[2,4]   w[2,5]      w[2,22]   w[2,23]   w[2,24]
 w[3,1]   w[3,2]   w[3,3]   w[3,4]   w[3,5]      w[3,22]   w[3,23]   w[3,24]
 w[4,1]   w[4,2]   w[4,3]   w[4,4]   w[4,5]      w[4,22]   w[4,23]   w[4,24]
 w[5,1]   w[5,2]   w[5,3]   w[5,4]   w[5,5]      w[5,22]   w[5,23]   w[5,24]
 w[6,1]   w[6,2]   w[6,3]   w[6,4]   w[6,5]   …  w[6,22]   w[6,23]   w[6,24]
 w[7,1]   w[7,2]   w[7,3]   w[7,4]   w[7,5]      w[7,22]   w[7,23]   w[7,24]
 w[8,1]   w[8,2]   w[8,3]   w[8,4]   w[8,5]      w[8,22]   w[8,23]   w[8,24]
 w[9,1]   w[9,2]   w[9,3]   w[9,4]   w[9,5]      w[9,22]   w[9,23]   w[9,24]
 w[10,1]  w[10,2]  w[10,3]  w[10,4]  w[10,5]     w[10,22]  w[10,23]  w[10,24]
 w[11,1]  w[11,2]  w[11,3]  w[11,4]  w[11,5]  …  w[11,22]  w[11,23]  w[11,24]
 w[12,1]  w[12,2]  w[12,3]  w[12,4]  w[12,5]     w[12,22]  w[12,23]  w[12,24]
 w[13,1]  w[13,2]  w[13,3]  w[13,4]  w[13,5]  

In [20]:
model.VariableRef

ErrorException: type GenericModel has no field VariableRef